<a href="https://colab.research.google.com/github/kookie14/sentimen-analysis/blob/main/linearsvc_lr_svc_bernoulli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install unidecode
!pip install emoji

In [ ]:
%%bash
pip install transformers
pip install fastBPE
pip install FAIRSeq

In [ ]:
!pip install unidecode

In [ ]:
import re
import pandas as pd
import string
import unidecode
import numpy as np
import scipy.sparse as sp
import emoji
from sklearn import utils as skutils
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
data = '/content/drive/MyDrive/Machine Learning/data_file.csv'

In [ ]:
df = pd.read_csv(data)
df.head()

In [ ]:
X, y = df['Comment'].values, df['Rating'].values

In [ ]:
X[7]

# Tiền xử lý 

* Chuẩn hoá về chữ thường

In [ ]:
def lowerCase(data):
    data = data.lower()
    return data

In [ ]:
test = 'Xôi dẻo, đồ ăn đậm vị. Hộp xôi được lót lá trông rất thích'
test = lowerCase(test)
test

* xoá dấu câu

In [ ]:
def remove_punctuations(data):
    punct_tag = re.compile(r'[^\w\s]')
    data = punct_tag.sub(r'',data)
    return data

In [ ]:
remove_punctuations(test)

* xoá thẻ html

In [ ]:
def remove_html(data):
    html_tag=re.compile(r'<.*?>')
    data=html_tag.sub(r'',data)
    return data

* xoá dữ liệu URL

In [ ]:
def remove_url(data):
    url_clean= re.compile(r"https://\S+|www\.\S+")
    data=url_clean.sub(r'',data)
    return data

* remove dấu xuống dòng

In [ ]:
def remove_n(data):
    html_tag=re.compile(r'\n')
    data=html_tag.sub(r' ',data)
    return data

* xoá icon, emoji

In [ ]:
def remove_emoji(data):
    emoji_clean= re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    data=emoji_clean.sub(r'',data)
    url_clean= re.compile(r"https://\S+|www\.\S+")
    data=url_clean.sub(r'',data)
    return data

In [ ]:
!pip install unidecode

In [ ]:

def removeTone(data):
  return unidecode.unidecode(data)

In [ ]:
def removeConsecutiveSpaces(data):
  return ' '.join(data.split())

In [ ]:
def remove_n(data):
    html_tag=re.compile(r'\n')
    data=html_tag.sub(r' ',data)
    return data

In [ ]:
def remove_Iter(text):
  text = re.sub(r'([A-Z])\1+', lambda m: m.group(1).upper(), text, flags=re.IGNORECASE)
  return text

In [ ]:
test = 'đẹppppp quasssss'
print(remove_Iter(test))

In [ ]:
df['Comment'] = df['Comment'].apply(lambda z: remove_punctuations(z))

df['Comment'] = df['Comment'].apply(lambda z: remove_n(z))

df['Comment'] = df['Comment'].apply(lambda z: remove_html(z))

df['Comment'] = df['Comment'].apply(lambda z: remove_url(z))

df['Comment'] = df['Comment'].apply(lambda z: remove_emoji(z))

df['Comment'] = df['Comment'].apply(lambda z: removeTone(z))

df['Comment'] = df['Comment'].apply(lambda z: removeConsecutiveSpaces(z))

df['Comment'] = df['Comment'].apply(lambda z: remove_n(z))

df['Comment'] = df['Comment'].apply(lambda z: remove_Iter(z))

df['Comment'] = df['Comment'].apply(lambda z: lowerCase(z))


In [ ]:
df.head()

In [ ]:
labels = []
for index, values in enumerate(y):
  if y[index] < 6: values = 0
  else: values = 1
  labels.append(values)

In [ ]:
clean_data = pd.DataFrame({ 'Comment':df['Comment'], 'Label': labels})
clean_data.to_csv('/content/drive/MyDrive/Machine Learning/clean_data.csv')

In [ ]:
clean_data.head()

In [ ]:
texts = clean_data['Comment'].values
label = clean_data['Label'].values

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import metrics

In [ ]:
X_train = clean_data['Comment'].values
y_train =  clean_data['Label'].values

In [ ]:
x_train, x_test, l_train, l_test = train_test_split(X_train,y_train, test_size=0.2, random_state=42)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(max_features=5000,stop_words='english')
vect

In [ ]:
X_dtm = vect.fit_transform(x_train)
test_X_dtm = vect.transform(x_test)

In [ ]:
labels = ['0', '1']
submit = []
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logreg = LogisticRegression(C=12.0)
for label in labels:
   print('... Processing {}'.format(label))
   logreg.fit(X_dtm, l_train)
   y_pre = logreg.predict(X_dtm)
   print('Training Accuracy is {}'.format(accuracy_score(l_train,y_pre)))
   test_y = logreg.predict(test_X_dtm)
   test_y_prob = logreg.predict(test_X_dtm)
   print('Training Accuracy is {}'.format(accuracy_score(l_test,test_y_prob)))

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.svm import LinearSVC
classifier =  LinearSVC(fit_intercept = True,multi_class='ovr', C=1)
step = []
step.append(('CountVectorizer', CountVectorizer(ngram_range=(1,4),max_df=0.5, min_df=5)))
step.append(('tfidf', TfidfTransformer(use_idf=False, sublinear_tf = True,norm='l2',smooth_idf=True)))
step.append(('classifiers',classifier))
clf = Pipeline(step)
clf.fit(X_train, y_train)

In [ ]:
test_file = '/content/drive/MyDrive/Machine Learning/test.csv'
test = pd.read_csv(test_file)
test.head()

In [ ]:
test['Comment'] = test['Comment'].apply(lambda z: remove_punctuations(str(z)))

test['Comment'] = test['Comment'].apply(lambda z: remove_n(z))

test['Comment'] = test['Comment'].apply(lambda z: remove_html(z))

test['Comment'] = test['Comment'].apply(lambda z: remove_url(z))

test['Comment'] = test['Comment'].apply(lambda z: remove_emoji(z))

test['Comment'] = test['Comment'].apply(lambda z: removeTone(z))

test['Comment'] = test['Comment'].apply(lambda z: removeConsecutiveSpaces(z))

test['Comment'] = test['Comment'].apply(lambda z: remove_n(z))

test['Comment'] = test['Comment'].apply(lambda z: remove_Iter(z))

test['Comment'] = test['Comment'].apply(lambda z: lowerCase(str(z)))

In [ ]:
test.head()

In [ ]:
text = test['Comment'].values
text = np.array(text)
text_vec = vect.transform(text)
for label in labels:
  print('... Processing {}'.format(label))
  logreg.fit(X_dtm, l_train)
  y_pre = logreg.predict(X_dtm)
  print('Training Accuracy is {}'.format(accuracy_score(l_train,y_pre)))
  test_y_prob = logreg.predict_proba()[:,1]
    

In [ ]:
y_test =  clf.predict(text)

In [ ]:
submit = pd.DataFrame({'RevId': test['RevId'], 'Rating': test_y_prob})
submit.to_csv('/content/drive/MyDrive/Machine Learning/submit412.csv')
submit[0:10]

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.svm import LinearSVC
x = LinearSVC(fit_intercept = True,multi_class='crammer_singer', C=1)
step1 = []
step1.append(('CountVectorizer', CountVectorizer(ngram_range=(1,4),max_df=0.5, min_df=5)))
step1.append(('tfidf', TfidfTransformer(use_idf=False, sublinear_tf = True,norm='l2',smooth_idf=True)))
step1.append(('classifiers',x))
clf = Pipeline(step1)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.svm import LinearSVC
LRmodel = LogisticRegression(C = 1, max_iter = 2000, n_jobs=-1)
step2 = []
step2.append(('CountVectorizer', CountVectorizer(ngram_range=(1,5),max_df=0.5, min_df=5)))
step2.append(('tfidf', TfidfTransformer(use_idf=False, sublinear_tf = True,norm='l2',smooth_idf=True)))
step2.append(('classifiers',LRmodel))
clf1 = Pipeline(step2)

In [ ]:
x_train, x_test, l_train, l_test = train_test_split(X_train,y_train, test_size=0.2, random_state=26105111)

In [ ]:
clf.fit(x_train, l_train)

In [ ]:
predicty = clf.predict(x_test) 
report1 = metrics.classification_report(l_test,predicty , labels=[1,0], digits=3)
print(report1)

In [ ]:
clf1.fit(x_train, l_train)

In [ ]:
predicty = clf1.predict(x_test) 
report2 = metrics.classification_report(l_test,predicty , labels=[1,0], digits=3)
print(report2)

In [ ]:
clf1.fit(X_train, y_train)

In [ ]:
predictLr = clf1.predict(text)

In [ ]:
submit1 = pd.DataFrame({'RevId': test['RevId'], 'Rating': predictLr})
submit1.to_csv('/content/drive/MyDrive/Machine Learning/submit4.csv')
submit1.head()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.svm import SVC
svm = SVC(kernel='linear', C=0.2175, class_weight=None, verbose=True)
step3 = []
step3.append(('CountVectorizer', CountVectorizer(ngram_range=(1,5),max_df=0.5, min_df=5)))
step3.append(('tfidf', TfidfTransformer(use_idf=False, sublinear_tf = True,norm='l2',smooth_idf=True)))
step3.append(('classifiers',svm))
clf2 = Pipeline(step3)

In [ ]:
clf2.fit(x_train, l_train)

In [ ]:
predict3 = clf2.predict(x_test) 
report3 = metrics.classification_report(l_test,predict3 , labels=[1,0], digits=3)
print(report3)